In [88]:
# encoding=utf-8
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow_addons.text import crf

class myModel(keras.Model):
    def __init__(self,time_steps,batch_size):
        super(myModel,self).__init__()
        self.time_steps = time_steps
        self.batch_size = batch_size

        self.fw_LSTM = layers.LSTM(units=100,activation='relu',return_sequences=True,go_backwards=False)
        self.bw_LSTM = layers.LSTM(units=100,activation='relu',return_sequences=True,go_backwards=True)
        self.BiLSTM = layers.Bidirectional(self.fw_LSTM,backward_layer=self.bw_LSTM,input_shape=(self.batch_size, self.time_steps)) #todo

    def call(self, inputs, training=None, mask=None):
        x = self.BiLSTM(inputs)
        return x

    def embedding_layer(self):
        pass # todo

batch_size = 2
time_steps = 5

x1 = np.array([[1,2,3,4,5],[3,1,5,2,1],[3,4,5,0,2],[5,3,4,0,1]],dtype='int32')
y1 = np.array([[0,0,1,2,3],
               [1,0,0,0,0],
               [1,2,3,0,0],
               [0,1,2,0,0]],dtype='int32')
lens = np.array([5,5,5,5])
model = myModel(batch_size,time_steps)




In [113]:
LSTM_dim = 50
tag_num = 4
fw_LSTM = layers.LSTM(units=LSTM_dim,return_sequences=True,go_backwards=False)
bw_LSTM = layers.LSTM(units=LSTM_dim,return_sequences=True,go_backwards=True)
# BiLSTM = layers.Bidirectional(fw_LSTM,backward_layer=bw_LSTM,input_shape=(4, 5)) #todo
BiLSTM = layers.Bidirectional(fw_LSTM,backward_layer=bw_LSTM) #todo

# x3 = tf.random.normal((2,5,1))
# x3
# x = BiLSTM(x)
if tf.test.is_gpu_available():
  with tf.device("/gpu:0"):
    train_loss = keras.metrics.Mean(name='train_loss')
    train_acc = keras.metrics.CategoricalAccuracy(name='train_acc')


    initializer = tf.keras.initializers.GlorotUniform()
    trans_p = tf.Variable(initializer([tag_num, tag_num]), trainable=True, name="transitions")

    w_init = tf.random_normal_initializer()
    w_proj = tf.Variable(w_init([100,tag_num]),trainable=True)
    b_init = tf.zeros_initializer()
    b_proj = tf.Variable(b_init([tag_num]),trainable=True)
    proj_layer = layers.Dense(tag_num)
    optimizer = keras.optimizers.Adam(learning_rate=0.01)
    for epoch in range(0,50):
        with tf.GradientTape(persistent=True) as tape:
            # x = tf.keras.layers.Embedding(6, 1, mask_zero=True)(x1)
            x = tf.expand_dims(x1,axis=-1)
            x = tf.cast(x,dtype='float')
            x = BiLSTM(x)
            # 全连接映射层
            pred = proj_layer(x)
            # pred = tf.matmul(x,w_proj) + b_proj
            # print(pred)

            # crf
            loss, trans_p = crf.crf_log_likelihood(inputs=pred,tag_indices=y1,transition_params=trans_p,sequence_lengths=lens)
            loss = tf.reduce_sum(loss)
            seq_tags, best_score = crf.crf_decode(potentials=pred,transition_params=trans_p, sequence_length=lens)
            # print(trans_p.shape)
        if epoch%10==0:
            print(epoch)
            print(seq_tags)
            # print(trans_p)
            # print(BiLSTM.trainable_variables[0][1])

        grads = tape.gradient(loss,[BiLSTM.trainable_variables,trans_p])
        grads2 = tape.gradient(loss,proj_layer.trainable_variables)
        # grads_crf = tape.gradient(loss,trans_p)
        grads_and_vars_clip0 = [[tf.clip_by_value(g, -5, 5), v] for g, v in zip(grads[0], BiLSTM.trainable_variables)]
        grads_and_vars_clip1 = [[tf.clip_by_value(g, -5, 5), v] for g, v in zip([grads[1]], [trans_p])]

        optimizer.apply_gradients(grads_and_vars_clip0)
        optimizer.apply_gradients(grads_and_vars_clip1)
        optimizer.apply_gradients(zip(grads2,proj_layer.trainable_variables))

    # train_loss.reset_states()
        train_acc.reset_states()
    # train_loss(loss)
        train_acc(y1,seq_tags)
        template = 'Epoch {}, train acc: {}'
        print (template.format(epoch+1,train_acc.result()*100))
    #                        train_loss.result()))




0
tf.Tensor(
[[2 0 1 0 1]
 [0 1 0 1 0]
 [0 1 0 1 0]
 [0 1 0 1 0]], shape=(4, 5), dtype=int32)
Epoch 1, train acc: 0.0
Epoch 2, train acc: 0.0
Epoch 3, train acc: 0.0
Epoch 4, train acc: 0.0
Epoch 5, train acc: 0.0
Epoch 6, train acc: 0.0
Epoch 7, train acc: 0.0
Epoch 8, train acc: 0.0
Epoch 9, train acc: 0.0
Epoch 10, train acc: 0.0
10
tf.Tensor(
[[2 3 2 3 2]
 [2 3 2 3 2]
 [2 3 2 3 2]
 [2 3 2 3 2]], shape=(4, 5), dtype=int32)
Epoch 11, train acc: 0.0
Epoch 12, train acc: 0.0
Epoch 13, train acc: 0.0
Epoch 14, train acc: 0.0
Epoch 15, train acc: 0.0
Epoch 16, train acc: 25.0
Epoch 17, train acc: 25.0
Epoch 18, train acc: 25.0
Epoch 19, train acc: 25.0
Epoch 20, train acc: 25.0
20
tf.Tensor(
[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]], shape=(4, 5), dtype=int32)
Epoch 21, train acc: 25.0
Epoch 22, train acc: 25.0
Epoch 23, train acc: 25.0
Epoch 24, train acc: 25.0
Epoch 25, train acc: 25.0
Epoch 26, train acc: 25.0
Epoch 27, train acc: 25.0
Epoch 28, train acc: 25.0
Epoch 29, tr

In [ ]:
loss_object = keras.losses.SparseCategoricalCrossentropy()
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name='train_loss')
train_acc = keras.metrics.SparseTopKCategoricalAccuracy(name='train_acc')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_acc = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')

def train_step(input, labels):
    with tf.GradientTape() as tape:
        predict = model(input)
        loss = loss_object(predict, y1)

In [48]:
x=tf.constant(1.0)
w=tf.constant(2.0)
y=tf.Variable(1.0,trainable=True)
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    z = w*y*y*y
    func1 = x*z
df_dx = tape.gradient(func1,x)
dz_dy = tape.gradient(func1,y)
grads = tape.gradient(func1,[x,y])
print(df_dx)
print(dz_dy)
print(grads)

tf.Tensor(2.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=2.0>, <tf.Tensor: shape=(), dtype=float32, numpy=6.0>]


In [1]:
import pickle
path = r'F:\zzd\毕业论文\论文代码\DataSets\2014人民日报\vocab_test.pkl'
with open(path, 'rb') as f:
    word2id = pickle.load(f)
print(type(word2id))
print(word2id['你'])

<class 'dict'>
635


In [33]:
import tensorflow as tf
import numpy as np
l = [[[1,2,3],[4,5,6]]]
p = tf.convert_to_tensor(l)
print(p)
s = np.array(p)
l = s.flatten()
lo = []
lo.append(l)
print(lo)
for i in lo:
    for l in i:
        print(l)

tf.Tensor(
[[[1 2 3]
  [4 5 6]]], shape=(1, 2, 3), dtype=int32)
[array([1, 2, 3, 4, 5, 6])]
1
2
3
4
5
6


In [6]:
l = 2.35625489154
print('number is {:.4f}'.format(l))

number is 2.3563


In [2]:
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import time
import os
x = [[1,2,3,4,5]]
y = [1,1,0,0,1]

LSTM_dim = 100
tag_num = 2
# 模型所需的层定义
embedding = layers.Embedding(input_dim=10000, output_dim=2, mask_zero=True)
dense = layers.Dense(tag_num)
# fw_LSTM = layers.LSTM(units=LSTM_dim, return_sequences=True, go_backwards=False)
# bw_LSTM = layers.LSTM(units=LSTM_dim, return_sequences=True, go_backwards=True)
# BiLSTM = layers.Bidirectional(fw_LSTM, backward_layer=bw_LSTM)
BiLSTM = layers.Bidirectional(layers.LSTM(100,return_sequences=True))

x = tf.convert_to_tensor(x,dtype='int32')
z = embedding(x)
z = BiLSTM(z)
z = dense(z)
print(z)

E:\Tools\Anaconda\envs\TF2.1\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


tf.Tensor(
[[[ 6.1379164e-04  2.7616858e-05]
  [ 6.5122318e-04  4.2452704e-04]
  [ 4.4814346e-04  6.5117230e-04]
  [ 3.3737579e-04  2.4767499e-04]
  [ 8.7461900e-04 -1.9915166e-04]]], shape=(1, 5, 2), dtype=float32)


In [15]:
from tensorflow_addons.text import crf

vocab_size = 10000
embedding_dim = 5
model = keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim),
    # layers.GlobalAveragePooling1D(),
    # layers.Dense(160, activation='relu'),
    # layers.Dense(1, activation='sigmoid')
    # layers.Dense(160, activation='relu'),
    layers.Bidirectional(layers.LSTM(2,return_sequences=True)),
    layers.Dense(2)
])
model.compile(optimizer=keras.optimizers.Adam(),
             loss=keras.losses.BinaryCrossentropy(),
             metrics=['accuracy'])
x = [[1,2,3,4,5],[5,4,3,2,1]]
y = [[1,1,1,1,0],[1,1,1,1,1]]
lens = [5,5]
x = tf.constant(x)
y = tf.constant(y)
# lens = tf.constant(lens)
# opt = keras.optimizers.Adam()
with tf.GradientTape(persistent=True) as tape:
    z = model.predict(x)
    loss,transparam = crf.crf_log_likelihood(z,y,lens)
    # loss = tf.reduce_mean(loss)
grads1 = tape.gradient(loss,model.variables)
# grads2 = tape.gradient(loss,transparam)
# opt.apply_gradients(zip(grads1,model.variables))
# opt.apply_gradients(zip(grads2,transparam))


ValueError: Tried to convert 'tensor' to a tensor and failed. Error: None values not supported.

In [37]:
x =tf.Variable(initial_value=2.)
y=tf.Variable(initial_value=3.)
with tf.GradientTape() as tape:
    z = 6*x
    p = z*y**2
grads = tape.gradient(p,[x,y])
print(grads)

[<tf.Tensor: shape=(), dtype=float32, numpy=54.0>, <tf.Tensor: shape=(), dtype=float32, numpy=72.0>]


In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
from tensorflow_addons.text import crf

tf.test.is_built_with_cuda()

E:\Tools\Anaconda\envs\TF2.1\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.2) or chardet (4.0.0) doesn't match a supported version!
  RequestsDependencyWarning)


True

In [ ]:
score = [[
    [1, 2, 3],
    [2, 1, 3],
    [1, 3, 2],
    [3, 2, 1],
    [1,1,1],

]]  # (batch_size, time_step, num_tabs)
transition = [
    [2, 1, 3],
    [1, 3, 2],
    [3, 2, 1]
]   # (num_tabs, num_tabs)
lengths = [3]   # (batch_size, time_step)

score_t         = tf.constant(score, dtype=tf.int64)
transition_t    = tf.constant(transition, dtype=tf.int64)
lengths_t       = tf.constant(lengths, dtype=tf.int64)

# r1 = crf.crf_decode(score1,m,ls)
r2 = crf.crf_decode(
    potentials=score_t,
    transition_params=transition_t,
    sequence_length=[4])
r3 = crf.viterbi_decode(np.array(score[0]),np.array(transition))
# r4 = crf.viterbi_decode(score2,m)
# print(r1)
print(r2)
print(r3)
# print(r4)
s = tf.required_space_to_batch_paddings

In [33]:
s = [[1,2,3],[4,5,6,7],[1,2,3,4,5,6,7,8,9]]
ls = [3,4,9]
print(ls +1)
p = keras.preprocessing.sequence.pad_sequences(s,padding='post',value=0,truncating='post')
p = keras.preprocessing.sequence.pad_sequences(padding='post',value=0,truncating='post')
print(p)

TypeError: can only concatenate list (not "int") to list

In [ ]:
{ "Success" : "10", "IDS" : [{ "downloadInfo": { "Success" : "1", "ID" : "170474", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "461.40", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2021-1-26", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/170474/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/461.40/461.40-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:0%20sTime:0", "DownloadURLFileSize" : "633.03 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGame%20Ready%20for%20The%20Medium%3C/strong%3E%3Cbr%20/%3EThis%20new%20Game%20Ready%20Driver%20provides%20support%20for%20The%20Medium%2C%20a%20new%20adventure-horror%20game%20that%20features%20ray%20tracing%20and%20our%20performance-accelerating%20NVIDIA%20DLSS%20technology.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGaming%20Technology%3C/strong%3E%3Cbr%20/%3EIncludes%20support%20for%20GeForce%20RTX%2030%20Series%20laptops%20%28RTX%203080%2C%20RTX%203070%2C%20RTX%203060%29%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/the-medium-game-ready-driver%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.40/461.40-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v461.40%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v461.40%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.40/461.40-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2030%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%203080%20Laptop%20GPU"},{"productName" : "GeForce%20RTX%203070%20Laptop%20GPU"},{"productName" : "GeForce%20RTX%203060%20Laptop%20GPU"}]},{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "170546", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "NVIDIA%20Studio%20Driver", "ShortDescription" : "NVIDIA%20Studio%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "461.40", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "0", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2021-1-26", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/170546/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/461.40/461.40-notebook-win10-64bit-international-nsd-dch-whql.exe", "EMAN_REVRES_BD" : "101-a1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:0%20sTime:0", "DownloadURLFileSize" : "633.18 MB", "BannerURL" : "https://www.nvidia.cn/geforce/billboards/displaydriver/standalone/eve-std-studio-driver-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.cn/geforce/billboards/displaydriver/gfe/eve-gfe-studio-driver-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3ENVIDIA%20Studio%20Drivers%20provide%20artists%2C%20creators%2C%20and%203D%20developers%20the%20best%20performance%20and%20reliability%20when%20working%20with%20creative%20applications.%20To%20achieve%20the%20highest%20level%20of%20reliability%2C%20Studio%20Drivers%20undergo%20extensive%20testing%20against%20multi-app%20creator%20workflows%20and%20multiple%20revisions%20of%20the%20top%20creative%20applications%20from%20Adobe%20to%20Autodesk%20and%20beyond.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EApplications%3C/strong%3E%3Cbr%20/%3E%3Cbr%20/%3EThe%20January%20NVIDIA%20Studio%20Driver%20provides%20optimal%20support%20for%20the%20latest%20creative%20applications%20including%20Affinity%20Photo%20%28now%20with%20GPU-acceleration%29%20and%20the%20NVIDIA%20Broadcast%201.1%20Update.%3Cbr%20/%3E%3Cbr%20/%3EIn%20addition%2C%20this%20release%20also%20provides%20support%20for%20the%20latest%20%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/studio/%22%3ENVIDIA%20Studio%3C/a%3E%20laptops%20powered%20by%20new%20GeForce%20RTX%2030%20Series%20GPUs.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/january-2021-nvidia-studio-driver/%22%3ELearn%20more%20about%20these%20new%20releases%20in%20our%20Studio%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%3Cp%3EExceptions%3A%20%3C/p%3E%0D%0A%3Col%3E%0D%0A%20%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%0D%0A%20%20%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%0D%0A%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%0D%0A%3C/ol%3E%0D%0A%3Cbr%20/%3E%0D%0ARelease%20Notes%3A%3Cbr%20/%3E%0D%0A%3Cul%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.40/461.40-win10-nsd-release-notes.pdf%22%20title%3D%22NVIDIA%20Studio%20Driver%20Release%20Notes%20v461.40%22%3ENVIDIA%20Studio%20Driver%20Release%20Notes%20%28v461.40%29%3C/a%3E%3C/li%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.40/461.40-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%0D%0A%3C/ul%3E%0D%0A","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2030%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%203080%20Laptop%20GPU"},{"productName" : "GeForce%20RTX%203070%20Laptop%20GPU"},{"productName" : "GeForce%20RTX%203060%20Laptop%20GPU"}]},{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "169142", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "461.09", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2021-1-7", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/169142/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/461.09/461.09-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:1%20sTime:0", "DownloadURLFileSize" : "630.87 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.09/461.09-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v461.09%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v461.09%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/461.09/461.09-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "167964", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "460.89", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-12-15", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/167964/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/460.89/460.89-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:0%20sTime:0", "DownloadURLFileSize" : "628.11 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGame%20Ready%20for%20Quake%20II%20RTX%20with%20Vulkan%20Ray%20Tracing%3C/strong%3E%3Cbr%20/%3EOur%20latest%20Game%20Ready%20Driver%20provides%20support%20for%20the%20Quake%20II%20RTX%20v1.4.0%20update%20which%20enables%20support%20for%20the%20new%20Vulkan%20Ray%20Tracing%20extensions.%20Now%2C%20any%20GPU%20with%20support%20for%20Vulkan%20Ray%20Tracing%20can%20experience%20Quake%20II%20RTX%20in%20all%20its%20path-traced%20glory.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/quake-ii-rtx-vulkan-ray-tracing-game-ready-driver/%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.89/460.89-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v460.89%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v460.89%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.89/460.89-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "168036", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "NVIDIA%20Studio%20Driver", "ShortDescription" : "NVIDIA%20Studio%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "460.89", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "0", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-12-15", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/168036/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/460.89/460.89-notebook-win10-64bit-international-nsd-dch-whql.exe", "EMAN_REVRES_BD" : "101-a1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:0%20sTime:1", "DownloadURLFileSize" : "627.94 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-studio-driver-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-studio-driver-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3ENVIDIA%20Studio%20Drivers%20provide%20artists%2C%20creators%2C%20and%203D%20developers%20the%20best%20performance%20and%20reliability%20when%20working%20with%20creative%20applications.%20To%20achieve%20the%20highest%20level%20of%20reliability%2C%20Studio%20Drivers%20undergo%20extensive%20testing%20against%20multi-app%20creator%20workflows%20and%20multiple%20revisions%20of%20the%20top%20creative%20applications%20from%20Adobe%20to%20Autodesk%20and%20beyond.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EApplications%3C/strong%3E%3Cbr%20/%3E%3Cbr%20/%3EThe%20December%20NVIDIA%20Studio%20Driver%20provides%20optimal%20support%20for%20the%20latest%20creative%20applications%20including%20Chaos%20Vantage%2C%20OctaneRender%202020.2%2C%20V-Ray%205%20Benchmark%2C%20Boris%20FX%20Sapphire%202021%2C%20Unreal%20Engine%204.26%2C%20and%20the%20NVIDIA%20Omniverse%20Open%20Beta.%3Cbr%20/%3E%3Cbr%20/%3EIn%20addition%2C%20this%20release%20also%20provides%20support%20for%20the%20new%20flagship%20professional%20RTX%20A6000%20GPU.%20Built%20on%20the%20revolutionary%20NVIDIA%20Ampere%20architecture%2C%20RTX%20A6000%20combines%20next-generation%20RT%20Cores%2C%20Tensor%20Cores%2C%20and%20CUDA%26reg%3B%20cores%20with%2048%20GB%20of%20graphics%20memory%20for%20unprecedented%20graphics%2C%20rendering%2C%20and%20AI%20performance.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/december-2020-nvidia-studio-driver/%22%3ELearn%20more%20about%20these%20new%20releases%20in%20our%20Studio%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%3Cp%3EExceptions%3A%20%3C/p%3E%0D%0A%3Col%3E%0D%0A%20%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%0D%0A%20%20%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%0D%0A%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%0D%0A%3C/ol%3E%0D%0A%3Cbr%20/%3E%0D%0ARelease%20Notes%3A%3Cbr%20/%3E%0D%0A%3Cul%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.89/460.89-win10-nsd-release-notes.pdf%22%20title%3D%22NVIDIA%20Studio%20Driver%20Release%20Notes%20v460.89%22%3ENVIDIA%20Studio%20Driver%20Release%20Notes%20%28v460.89%29%3C/a%3E%3C/li%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.89/460.89-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%0D%0A%3C/ul%3E%0D%0A","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "167551", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "460", "Version" : "460.79", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-12-9", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/167551/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/460.79/460.79-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:0%20sTime:0", "DownloadURLFileSize" : "625.63 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGame%20Ready%20for%20Cyberpunk%202077%3C/strong%3E%3Cbr%20/%3EThis%20new%20Game%20Ready%20Driver%20provides%20the%20ultimate%20gaming%20experience%20for%20Cyberpunk%202077.%20In%20addition%2C%20this%20driver%20provides%20optimal%20day-1%20support%20for%20the%20official%20launch%20of%20Minecraft%20with%20RTX%20for%20Windows%2010.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/cyberpunk-2077-game-ready-driver%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.79/460.79-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v460.79%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v460.79%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/460.79/460.79-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "167238", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "455", "Version" : "457.51", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-12-2", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/167238/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/457.51/457.51-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:1%20sTime:0", "DownloadURLFileSize" : "619.44 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGaming%20Technology%3C/strong%3E%3Cbr%20/%3EIncludes%20support%20for%20the%20GeForce%20RTX%203060%20Ti%20GPU%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/geforce-rtx-3060-ti-game-ready-driver%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.51/457.51-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v457.51%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v457.51%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.51/457.51-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "166741", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "455", "Version" : "457.30", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-11-9", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/166741/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/457.30/457.30-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:1%20sTime:0", "DownloadURLFileSize" : "617.69 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGame%20Ready%20for%20Call%20of%20Duty%3A%20Black%20Ops%20Cold%20War%3C/strong%3E%3Cbr%20/%3EThis%20new%20Game%20Ready%20Driver%20provides%20support%20for%20Call%20of%20Duty%3A%20Black%20Ops%20Cold%20War.%20In%20addition%2C%20this%20driver%20provides%20optimal%20day-1%20support%20for%20Assassin%26rsquo%3Bs%20Creed%20Valhalla%2C%20and%20Godfall.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/call-of-duty-black-ops-cold-war-game-ready-driver%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.30/457.30-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v457.30%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v457.30%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.30/457.30-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "167015", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "NVIDIA%20Studio%20Driver", "ShortDescription" : "NVIDIA%20Studio%20Driver", "DeviceToProductFamilyName" : "", "Release" : "455", "Version" : "457.30", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "0", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-11-17", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/167015/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/457.30/457.30-notebook-win10-64bit-international-nsd-dch-whql.exe", "EMAN_REVRES_BD" : "101-a1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:2%20sTime:0", "DownloadURLFileSize" : "617.86 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-studio-driver-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-studio-driver-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3ENVIDIA%20Studio%20Drivers%20provide%20artists%2C%20creators%2C%20and%203D%20developers%20the%20best%20performance%20and%20reliability%20when%20working%20with%20creative%20applications.%20To%20achieve%20the%20highest%20level%20of%20reliability%2C%20Studio%20Drivers%20undergo%20extensive%20testing%20against%20multi-app%20creator%20workflows%20and%20multiple%20revisions%20of%20the%20top%20creative%20applications%20from%20Adobe%20to%20Autodesk%20and%20beyond.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EApplications%3C/strong%3E%3Cbr%20/%3E%3Cbr%20/%3EThe%20November%20NVIDIA%20Studio%20Driver%20provides%20optimal%20support%20for%20the%20latest%20creative%20applications%20including%20the%20DaVinci%20Resolve%2017%20Beta%2C%20Keyshot%2010%2C%20Notch%2C%20Topaz%20Video%20Enhance%20AI%2C%20BorisFX%20Continuum%202021%2C%20Red%20Giant%20Trapcode%20Suite%2016%2C%20and%20Magic%20Bullet%20Suite%2014.%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/november-2020-nvidia-studio-driver/%22%3ELearn%20more%20about%20these%20new%20releases%20in%20our%20Studio%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%3Cp%3EExceptions%3A%20%3C/p%3E%0D%0A%3Col%3E%0D%0A%20%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%0D%0A%20%20%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%0D%0A%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%0D%0A%3C/ol%3E%0D%0A%3Cbr%20/%3E%0D%0ARelease%20Notes%3A%3Cbr%20/%3E%0D%0A%3Cul%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.30/457.30-win10-nsd-release-notes.pdf%22%20title%3D%22NVIDIA%20Studio%20Driver%20Release%20Notes%20v457.30%22%3ENVIDIA%20Studio%20Driver%20Release%20Notes%20%28v457.30%29%3C/a%3E%3C/li%3E%0D%0A%20%20%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.30/457.30-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%0D%0A%3C/ul%3E%0D%0A","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]}]}},{ "downloadInfo": { "Success" : "1", "ID" : "166485", "DownloadTypeID" : "1", "DownloadStatusID" : "3", "Name" : "GeForce%20Game%20Ready%20Driver", "ShortDescription" : "GeForce%20Game%20Ready%20Driver", "DeviceToProductFamilyName" : "", "Release" : "455", "Version" : "457.09", "DisplayVersion" : "", "CDKitUSBEmitterDriverVersion" : "", "CDKitGPUDriverVersion" : "", "IsBeta" : "0", "IsWHQL" : "1", "IsArchive" : "0", "IsRecommended" : "0", "IsActive" : "1", "IsNewest" : "0", "IsEmailRequired" : "0", "ReleaseDateTime" : "2020-10-29", "DetailsURL" : "http://www.nvidia.cn/Download/driverResults.aspx/166485/cn", "DownloadURL" : "https://cn.download.nvidia.com/Windows/457.09/457.09-notebook-win10-64bit-international-dch-whql.exe", "EMAN_REVRES_BD" : "101-b1espa-lqssmrp-lswfg", "EMITR_BD" : "memCache:1%20rTime:1%20sTime:0", "DownloadURLFileSize" : "611.05 MB", "BannerURL" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/standalone/eve-std-ces-gsync-fwg-jan20-banners/", "BannerURLGfe" : "https://www.nvidia.com/zh-cn/geforce/billboards/displaydriver/gfe/eve-gfe-ces-gsync-fwg-jan20-banners/", "ReleaseNotes":"As%20part%20of%20the%20%3Ca%20href%3D%22https%3A//www.nvidia.com/Download/index.aspx%3Flang%3Den-us%22%3ENVIDIA%20Notebook%20Driver%20Program%3C/a%3E%2C%20this%20is%20a%20reference%20driver%20that%20can%20be%20installed%20on%20supported%20NVIDIA%20notebook%20GPUs.%20However%2C%20please%20note%20that%20your%20notebook%20original%20equipment%20manufacturer%20%28OEM%29%20provides%20certified%20drivers%20for%20your%20specific%20notebook%20on%20their%20website.%20NVIDIA%20recommends%20that%20you%20check%20with%20your%20notebook%20OEM%20about%20recommended%20software%20updates%20for%20your%20notebook.%20OEMs%20may%20not%20provide%20technical%20support%20for%20issues%20that%20arise%20from%20the%20use%20of%20this%20driver.%3Cbr%20/%3E%3Cbr%20/%3E%3Cb%3EBefore%20downloading%20this%20driver%3A%3C/b%3E%3Cbr%20/%3E%3Cbr%20/%3E%3Col%3E%3Cli%3EIt%20is%20recommended%20that%20you%20backup%20your%20current%20system%20configuration.%20%3Cu%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/drivers/notebook-driver-instructions/%22%3EClick%20here%20for%20instructions%3C/a%3E.%3C/u%3E%3C/li%3E%20%3C%21--%20%3Cli%3EIf%20you%20own%20a%20Dell%20Inspiron%201420%2C%20Dell%20XPS%20M1330%2C%20Dell%20XPS%20M1530%2C%20or%20Dell%20Latitude%20D630/D630c%20it%20is%20highly%20recommended%20that%20you%20first%20%3Cu%3E%3Ca%20href%3D%22http%3A//www.nvidia.com/object/recommended_software_updates.html%22%3Einstall%20this%20Dell%20software%20update%3C/a%3E%3C/u%3E.%3C/li%3E--%3E%3C/ol%3E%3Cbr%20/%3E%3Cbr%20/%3EGame%20Ready%20Drivers%20provide%20the%20best%20possible%20gaming%20experience%20for%20all%20major%20new%20releases.%20Prior%20to%20a%20new%20title%20launching%2C%20our%20driver%20team%20is%20working%20up%20until%20the%20last%20minute%20to%20ensure%20every%20performance%20tweak%20and%20bug%20fix%20is%20included%20for%20the%20best%20gameplay%20on%20day-1.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGame%20Ready%20for%20Watch%20Dogs%3A%20Legion%3C/strong%3E%3Cbr%20/%3EThis%20new%20Game%20Ready%20Driver%20provides%20support%20for%20Watch%20Dogs%3A%20Legion.%20In%20addition%2C%20this%20driver%20provides%20optimal%20day-1%20support%20for%20DiRT%205%2C%20Ghostrunner%2C%20Need%20for%20Speed%3A%20Hot%20Pursuit%20Remastered%2C%20and%20Xuan-Yuan%20Sword%20VII.%3Cbr%20/%3E%3Cbr%20/%3E%3Cstrong%3EGaming%20Technology%3C/strong%3E%3Cbr%20/%3EIncludes%20support%20for%20the%20GeForce%20RTX%203070%20GPU%3Cbr%20/%3E%3Cbr%20/%3E%3Ca%20href%3D%22https%3A//www.nvidia.com/en-us/geforce/news/watch-dogs-legion-rtx-3070-game-ready-driver%22%3ELearn%20more%20in%20our%20Game%20Ready%20Driver%20article%20here.%3C/a%3E","OtherNotes":"%09Please%20note%3A%20%20Effective%20April%2012%2C%202018%2C%20Game%20Ready%20Driver%20upgrades%2C%20including%20performance%20%20enhancements%2C%20new%20features%2C%20and%20bug%20fixes%2C%20will%20be%20available%20only%20for%20%20desktop%20Kepler%2C%20Maxwell%2C%20Pascal%2C%20Volta%2C%20Turing%20and%20Ampere-series%2C%20as%20well%20as%20for%20%20systems%20utilizing%20mobile%20Maxwell%2C%20Pascal%2C%20and%20Turing-series%20GPUs%20for%20notebooks.%3Cbr%20/%3E%3Cbr%20/%3E%20%20Following%20the%20posting%20%20of%20the%20final%20driver%20from%20Release%20418%20on%20April%2011%2C%202019%20GeForce%20Game%20Ready%20%20Drivers%20will%20no%20longer%20support%20NVIDIA%203D%20Vision%20or%20systems%20utilizing%20mobile%20%20Kepler-series%20GPUs.%20Critical%20security%20updates%20will%20be%20available%20for%20these%20%20products%20through%20April%202020.%20A%20complete%20list%20of%20Kepler-series%26nbsp%3BGeForce%20GPUs%20%20can%20be%20found%26nbsp%3B%3Ca%20href%3D%22http%3A//nvidia.custhelp.com/app/answers/detail/a_id/4780%22%20target%3D%22_blank%22%3Ehere%3C/a%3E.%20%3Cbr%3E%3Cbr%3E%20%20%20%20Those%20looking%20to%20%20utilize%203D%20Vision%20can%20remain%20on%20a%20Release%20418%20driver.%20Our%20software%20that%20enables%20%20the%20use%20of%203D%20gaming%20with%203D%26nbsp%3BTVs%2C%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/object/3dtv-play-overview.html%22%3E3DTV%20Play%3C/a%3E%2C%26nbsp%3Bis%20now%20%20included%20for%20free%20in%20Release%20418.%20%26nbsp%3BIt%20is%20no%20longer%20available%20as%20a%20%20standalone%20download.%20%26nbsp%3BOur%26nbsp%3B%3Ca%20href%3D%22https%3A//www.nvidia.com/download/driverResults.aspx/128699/en-us%22%3E3D%20Vision%20Video%20%20Player%3C/a%3E%26nbsp%3Bwill%20%20continue%20to%20be%20offered%20as%20a%20standalone%20download%2C%20for%20free%2C%20until%20the%20end%20of%20%202019.%3Cbr%20/%3E%3Cbr%20/%3E%20%20%20%20Exceptions%3A%20%3Col%3E%20%3Cli%3ENotebooks%20supporting%20Hybrid%20Power%20technology%20are%20not%20supported%20%28NVIDIA%20Optimus%20technology%20is%20supported%29.%3C/li%3E%3Cli%3EThe%20following%20Sony%20VAIO%20notebooks%20are%20included%20in%20the%20Verde%20notebook%20program%3A%20Sony%20VAIO%20F%20Series%20with%20NVIDIA%20GeForce%20310M%2C%20GeForce%20GT%20330M%2C%20GeForce%20GT%20425M%2C%20GeForce%20GT%20520M%20or%20GeForce%20GT%20540M.%20Other%20Sony%20VAIO%20notebooks%20are%20not%20included%20%28please%20contact%20Sony%20for%20driver%20support%29.%3C/li%3E%20%20%3Cli%3EFujitsu%20notebooks%20are%20not%20included%20%28Fujitsu%20Siemens%20notebooks%20are%20included%29.%3C/li%3E%3C/ol%3E%3Cbr%20/%3ERelease%20Notes%3A%3Cbr%20/%3E%3Cul%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.09/457.09-win10-win8-win7-release-notes.pdf%22%20title%3D%22Game%20Ready%20Driver%20Release%20Notes%20v457.09%22%3EGame%20Ready%20Driver%20Release%20Notes%20%28v457.09%29%3C/a%3E%3Cli%3E%3Ca%20href%3D%22https%3A//us.download.nvidia.com/Windows/457.09/457.09-nvidia-control-panel-quick-start-guide.pdf%22%20title%3D%22Control%20Panel%20User%27s%20Guide%22%3EControl%20Panel%20User%27s%20Guide%3C/a%3E%20%3C/li%3E%3C/ul%3E%20%20%20%20%20%20%09","InstallationNotes":"","Overview":"","LanguageName" : "Chinese%20(Simplified)", "OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0", "OSList":[{"OSName" : "Windows%2010%2064-bit", "OsCode" : "10.0"}],"Is64Bit" : "1", "Messaging" : [{"MessageCode" : "DownloadIDDetailsFound", "MessageValue" : "The Driver DownloadID details found"}], "series" : [{"seriesname" : "GeForce%20RTX%2020%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20RTX%202080%20SUPER"},{"productName" : "GeForce%20RTX%202080"},{"productName" : "GeForce%20RTX%202070%20SUPER"},{"productName" : "GeForce%20RTX%202070"},{"productName" : "GeForce%20RTX%202060"}]},{"seriesname" : "GeForce%20MX400%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX450"}]},{"seriesname" : "GeForce%20MX300%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX350"},{"productName" : "GeForce%20MX330"}]},{"seriesname" : "GeForce%20MX200%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20MX250"},{"productName" : "GeForce%20MX230"}]},{"seriesname" : "GeForce%20MX100%20Series%20(Notebook)", "products" : [{"productName" : "GeForce%20MX150"},{"productName" : "GeForce%20MX130"},{"productName" : "GeForce%20MX110"}]},{"seriesname" : "GeForce%20GTX%2016%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201660%20Ti"},{"productName" : "GeForce%20GTX%201650%20Ti"},{"productName" : "GeForce%20GTX%201650"}]},{"seriesname" : "GeForce%2010%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%201080"},{"productName" : "GeForce%20GTX%201070"},{"productName" : "GeForce%20GTX%201060"},{"productName" : "GeForce%20GTX%201050%20Ti"},{"productName" : "GeForce%20GTX%201050"}]},{"seriesname" : "GeForce%20900M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20980"},{"productName" : "GeForce%20GTX%20980M"},{"productName" : "GeForce%20GTX%20970M"},{"productName" : "GeForce%20GTX%20965M"},{"productName" : "GeForce%20GTX%20960M"},{"productName" : "GeForce%20GTX%20950M"},{"productName" : "GeForce%20945M"},{"productName" : "GeForce%20940MX"},{"productName" : "GeForce%20930MX"},{"productName" : "GeForce%20920MX"},{"productName" : "GeForce%20940M"},{"productName" : "GeForce%20930M"}]},{"seriesname" : "GeForce%20800M%20Series%20(Notebooks)", "products" : [{"productName" : "GeForce%20GTX%20860M"},{"productName" : "GeForce%20GTX%20850M"},{"productName" : "GeForce%20845M"},{"productName" : "GeForce%20840M"},{"productName" : "GeForce%20830M"}]}]}}] ,"spcall" : "EXEC NDL_usp_Download_select_by_matrix_values null,820,102,null,57,null,0, null,2052,-1,null,0,null,null,null,null,0,null,null,0,null,10,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,1,null", "Request" : [{"psid" : "102", "pfid" : "820", "osID" : "57", "languageCode" : "2052", "beta" : "0", "isWHQL" : "0", "dltype" : "-1", "sort1" : "0", "numberOfResults" : "10"}]}